Run the sections below if you want to do the google lookups on the original sales data

In [ ]:
# Careful. This costs $$$
GOOGLE_API_KEY='GOOGLE_API_KEY'
if GOOGLE_API_KEY not in os.environ:
    raise Exception(f"Must set {GOOGLE_API_KEY} environment variable to call Google's APIs.")

In [ ]:
gmaps = googlemaps.Client(os.environ[GOOGLE_API_KEY])

In [ ]:
customers = pd.read_csv(CUSTOMERS_FILE)
customers

In [ ]:
latlong=customers.Address.map(gmaps.geocode)
latlong

In [ ]:
#[response for response in latlong] #['geometry']['location']['lat']
def get_latlong(response, field):
    try:
        r = response[0]['geometry']['location'][field]
        return(r)
    except:
        print("Error: No Lat/Long Data, returning zero")
        return(None)

In [ ]:
customers['lat'] = [get_latlong(response, 'lat') for response in latlong]
customers['lng'] = [get_latlong(response, 'lng') for response in latlong]
customers

In [ ]:
import folium

def origin_marker(lat, lng):
    return folium.Marker(location=[lat, lng],
                         icon=folium.Icon(icon='star', color='blue'))

def customer_marker(lat, lng, bags):
    return folium.Marker(location=[lat, lng],
                         tooltip=str(bags),
                         icon=folium.Icon(icon='user', color='green'))

map = folium.Map(location=[origin_lat, origin_lng], default_zoom_start=17)
origin_marker(origin_lat, origin_lng).add_to(map)

for _, row in customers.iterrows():
    customer_marker(row.lat, row.lng, row.Bags).add_to(map)
map

In [ ]:
#We're limited to 25 addresses at a time per the Google Maps API
import math
addresses = customers.Address
response = list()
for i in range(math.ceil(len(addresses)/25)):
    lookup_addresses = addresses[i*25:(i+1)*25]
    gmaps_response = gmaps.distance_matrix(origins=[origin],
                                 destinations=lookup_addresses,
                                 mode='driving',
                                 avoid='tolls')
    response.append(gmaps_response)


In [ ]:
distances = list()
duration = list()
for d_set in response:
    for d in d_set['rows'][0]['elements']:
        if d['status'] == 'OK':
            distances.append(d['distance']['value'])
            duration.append(d['duration']['value'])
        else:
            distances.append(None)
            duration.append(None)

customers['distance'] = distances
customers['duration'] = duration

In [ ]:
#Saving Data after Google API lookup so we don't have to keep paying for the API
customers.to_csv(DATA_DIRECTORY + 'customer_data_latlong_timedistance.csv')

Start here if you don't want to re-run the Google API Lookup

In [1]:
import googlemaps
import os

# Riverbend Church
origin = '4214 N Capital of Texas Hwy, Austin, TX 78746'
origin_lat = 30.3418005
origin_lng = -97.8040886

import pandas as pd
DATA_DIRECTORY = '/Users/sorensen/Dropbox/Scouts/Mulch_Data_2019/'
#NOTE: Before Saving the .csv of Sales, make sure you do the following from the original spreadsheet:
#1.  Ensure the Address, Bags, ... fields are formattted correctly
#2.  Remove any non-address entries from the "Address" column as they are not to be delivered
CUSTOMERS_FILE = DATA_DIRECTORY + 'Sales.csv'

In [2]:
customers = pd.read_csv(DATA_DIRECTORY + 'customer_data_latlong_timedistance.csv')
customers = customers[customers.Bags > 0]
vehicles = pd.read_csv(DATA_DIRECTORY + 'Vehicles.csv').sort_values('Capacity', ascending=False)
#Initial load will have 137 customers

In [54]:
class Scheduler:
#Base class for mulch delivery Scheduling
    def __init__(self, customer_data, vehicle_data):
        self.customers = customer_data
        self.vehicles = vehicle_data
        
    def generate_schedule(self):
        return(None)

class MS_Scheduler_v1(Scheduler):

    def get_first_delivery(self, vehicle_capacity, delivery_data):
        available_delivery_data = delivery_data[delivery_data.Bags <= vehicle_capacity].copy()
        available_delivery_data['bags_per_second'] = available_delivery_data.Bags/delivery_data.duration
        available_delivery_data = available_delivery_data.sort_values('bags_per_second', ascending = False)
        available_delivery_data = available_delivery_data.reset_index(drop=True)
        #Pick our initial address and remove it from the delivery data
        first_delivery = dict(available_delivery_data.iloc[0])
        remaining_capacity = vehicle_capacity - first_delivery['Bags']
        #remaining_deliveries = delivery_data.drop([0])
        #return our first delivery location, the remaining deliveries, and the remaining vehicle capacity
        return(first_delivery, remaining_capacity)

    def get_next_delivery(self, vehicle_capacity, delivery_data, previous_delivery):
        #Remove any over-capacity locations

        available_delivery_data = delivery_data[delivery_data.Bags <= vehicle_capacity].copy()
        if available_delivery_data.empty:
            logging.debug("Vehicle at capacity")
            #No deliveries remain that we can accomodate, return None
            return(None)

        #Figure out the bags per distance
        #This is problematic and is generating a warning for setting a value on a copy of a slice
        #remaining_deliveries['bags_per_distance'] 
        available_delivery_data['bags_per_distance'] = (available_delivery_data.Bags/
                                              (abs(available_delivery_data.lat - previous_delivery['lat']) + 
                                               abs(available_delivery_data.lng - previous_delivery['lng'])))
        #Idea for the threshold... its bags per distance of going back to origin and taking next largest order
        #hard coding this for now... need to fix later
        origin_lat = 30.3418005
        origin_lng = -97.8040886
        bpd_return_to_origin = (self.customers_in_queue.Bags/
                                    (abs(previous_delivery['lat'] - origin_lat) +
                                      abs(previous_delivery['lng'] - origin_lng) +
                                      abs(self.customers_in_queue.lat - origin_lat) +
                                      abs(self.customers_in_queue.lng - origin_lng)))

        max_bpd_on_return = max(bpd_return_to_origin)
        avg_bpd_on_return = (sum(bpd_return_to_origin)/len(bpd_return_to_origin))
        logging.debug("Max BPD on Return : %s" % max_bpd_on_return)
        logging.debug("Min BPD on Return : %s" % min(bpd_return_to_origin))
        logging.debug("Avg BPD on Return : %s" % avg_bpd_on_return)
        
        available_delivery_data = available_delivery_data.sort_values('bags_per_distance', ascending=False).reset_index(drop=True)

        next_delivery = dict(available_delivery_data.iloc[0])
        #remaining_deliveries = remaining_deliveries.drop([0])
        logging.debug("BPD on Next Delivery : " +str(next_delivery['bags_per_distance']))
        if next_delivery['bags_per_distance'] < avg_bpd_on_return:
            return(None)
        else:
            remaining_capacity = vehicle_capacity - next_delivery['Bags']
            return(next_delivery, remaining_capacity)

    def get_driver_deliveries(self, vehicle_data):
        deliveries = list()
        logging.debug("Delivery Targets at Start : %s" % self.customers_in_queue.shape[0])
        #Get the first delivery from Riverbend
        result = self.get_first_delivery(vehicle_data['Capacity'], self.customers_in_queue)    
        
        if result is None:
            #logging.info("No More available devlieries for " + vehicle_data['Driver'])
            return(None)
        else:
            (previous_delivery, remaining_capacity) = result
            deliveries.append(previous_delivery)
            delivery_index = self.customers_in_queue[self.customers_in_queue.Address == previous_delivery['Address']].index
            self.customers_in_queue = self.customers_in_queue.drop(delivery_index)

        logging.debug("Delivery Targets after First delivery : %s" % self.customers_in_queue.shape[0])
        
        #Get the subsequent deliveries
        while remaining_capacity > 0:
            result = self.get_next_delivery(remaining_capacity, self.customers_in_queue, previous_delivery)
            if result is None:
                #print("No More available devlieries for " + vehicle_data['Driver'])
                return(deliveries)
            else:
                (previous_delivery, remaining_capacity) = result
                deliveries.append(previous_delivery)
                delivery_index = self.customers_in_queue[self.customers_in_queue.Address == previous_delivery['Address']].index
                self.customers_in_queue = self.customers_in_queue.drop(delivery_index)
                logging.debug("Delivery Targets after next delivery : %s" % self.customers_in_queue.shape[0])

        return(deliveries)
    
    def print_delivery_schedule(self, ds): 
        for k in ds.keys():
            print(k)
            for l in ds[k]:
                print("Deliver {Bags} bags to {Customer} @ {Address}".format(**l))
                #print("Deliver " + str(l['Bags']) + ' to ' + 'Customerl['Address'])
        return(None)
    
    def generate_schedule(self):
        vehicle_data_sorted = self.vehicles.sort_values('Capacity', ascending = False).copy()
        #Copy our customers into an attribute that represents the customers in the queue
        self.customers_in_queue = customers.copy()
        logging.debug("----Starting Schedule Generation-------")
        deliveries = dict()
        j = 0
        #This is a stupid approach but ultimately works
        #Next version, get delivery times and operate this assuming the vehicle order changed up
        while not self.customers_in_queue.empty:
            j = j + 1
            for i in range(vehicle_data_sorted.shape[0]):
                if self.customers_in_queue.empty:
                    return(deliveries)
                deliveries[vehicle_data_sorted.iloc[i].Driver + "_" + str(j)] = self.get_driver_deliveries(vehicle_data_sorted.iloc[i])
        
        return(deliveries)

In [56]:
import logging
logging.basicConfig(filename='schedule_generation.log',level=logging.DEBUG)

customers = pd.read_csv(DATA_DIRECTORY + 'customer_data_latlong_timedistance.csv')
customers = customers[customers.Bags > 0]
vehicles = pd.read_csv(DATA_DIRECTORY + 'Vehicles.csv').sort_values('Capacity', ascending=False)

ms_sched_v1 = MS_Scheduler_v1(customers,vehicles)
my_sched_2 = ms_sched_v1.generate_schedule()
#print_delivery_schedule(deliveries)

In [60]:
my_sched_2['John Martin_2']

[{'Unnamed: 0': 110,
  'Scout': 'Jack Martin',
  'Customer': 'Bonnie Martin',
  'Address': "5905 Hayden's Cove",
  'Zip': 78730.0,
  'Bags': 65,
  'Paid (Dave)': 325.0,
  'Donation only': nan,
  'Paid (Dave) Date': '5-Mar',
  'Paid (JB)': 'x',
  'Paid (JB) Date': '5-Mar',
  'Neighborhood': "near Martin's",
  'Comments': 'Leave pallet in Driveway',
  'Out for Delivery': 'JS - Trailer - 10am',
  'lat': 30.372656699999997,
  'lng': -97.8233257,
  'distance': 7663.0,
  'duration': 543.0,
  'bags_per_second': 0.11970534069981584},
 {'Unnamed: 0': 109,
  'Scout': 'Jack Martin',
  'Customer': 'Judy Poranaes',
  'Address': "6000 Maury's Trail",
  'Zip': 78730.0,
  'Bags': 20,
  'Paid (Dave)': 100.0,
  'Donation only': nan,
  'Paid (Dave) Date': '5-Mar',
  'Paid (JB)': 'x',
  'Paid (JB) Date': '5-Mar',
  'Neighborhood': "near Martin's",
  'Comments': 'By basketball goal',
  'Out for Delivery': 'AS - trailer - noon',
  'lat': 30.373142699999995,
  'lng': -97.8241754,
  'distance': 8075.0,
  'dur

In [62]:
import folium

def origin_marker(lat, lng):
    return folium.Marker(location=[lat, lng],
                         icon=folium.Icon(icon='star', color='blue'))

def customer_marker(lat, lng, bags):
    return folium.Marker(location=[lat, lng],
                         tooltip=str(bags),
                         icon=folium.Icon(icon='user', color='green'))

def delivery_map(deliveries):
    delivery_map = folium.Map(location=[origin_lat, origin_lng], default_zoom_start=17)
    #origin_marker(origin_lat, origin_lng).add_to(map)

    for d in deliveries:
        customer_marker(d['lat'], d['lng'], d['Bags']).add_to(delivery_map)
    
    return(delivery_map)

delivery_map(my_sched_2['John Martin_4'])